In [1]:
import subprocess
import sys

packages = ['nltk', 'spacy', 'unidecode', 'numpy', 'pandas']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("Bibliothèques installées avec succès")

✓ Bibliothèques installées avec succès


In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import spacy
from pathlib import Path

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

try:
    nlp = spacy.load('fr_core_news_sm')
except OSError:
    print("Téléchargement du modèle spaCy français...")
    subprocess.check_call([sys.executable, '-m', 'spacy', 'download', 'fr_core_news_sm'])
    nlp = spacy.load('fr_core_news_sm')

print("Imports et modèles chargés")

Téléchargement du modèle spaCy français...
Imports et modèles chargés


In [19]:
def load_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return ""
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {e}")
        return ""

text = load_text_file('entree.txt')
print(f"Texte chargé: {len(text)} caractères")
print("\n--- Premier 200 caractères ---")
print(text[:200] + "...")

Texte chargé: 1616 caractères

--- Premier 200 caractères ---
OH OH OH, Noel arrive, et qui dit Noel dit ... nourriture !
Jeudi 18 décembre prochain, à partir de midi, nous vous proposons un grand buffet collectif de noel pour fêter ensemble vos départs en vacan...


In [26]:
def tokenize_words(text):
    print(f"Ressource NLTK manquante")
    import re
    tokens = re.findall(r'\b\w+\b|[.,!?;:\-]', text)
    return tokens

words = tokenize_words(text)
print(f"Nombre de mots: {len(words)}")
print(f"Premiers 20 mots: {words[:20]}")

Ressource NLTK manquante
Nombre de mots: 316
Premiers 20 mots: ['OH', 'OH', 'OH', ',', 'Noel', 'arrive', ',', 'et', 'qui', 'dit', 'Noel', 'dit', '.', '.', '.', 'nourriture', '!', 'Jeudi', '18', 'décembre']


In [16]:
def tokenize_sentences(text):
    sentences = re.split(r'\.\.\.|[.!?]\s*', text)

    return sentences

sentences = tokenize_sentences(text)
print(f"Nombre de phrases: {len(sentences)}")
print("\n--- Premières 3 phrases ---")
for i, sent in enumerate(sentences[:3], 1):
    print(f"{i}. {sent}")

Nombre de phrases: 13

--- Premières 3 phrases ---
1. OH OH OH, Noel arrive, et qui dit Noel dit 
2.  nourriture 
3. Jeudi 18 décembre prochain, à partir de midi, nous vous proposons un grand buffet collectif de noel pour fêter ensemble vos départs en vacances d'hiver après un premier semestre intense 


In [ ]:
def tokenize_bigrams(text):
    ngrams = []
    trigrammes = re.findall(r'(\w+)-(\w+)-(\w+)', text)
    for tri in trigrammes:
        ngrams.append(tri)  
    
    bigrammes = re.findall(r'(\w+)-(\w+)(?!-)', text)
    for bi in bigrammes:
        ngrams.append(bi) 
    
    return ngrams

bigrams = tokenize_bigrams(text)
print(f"Nombre de bigrammes/trigrammes (séparés par -): {len(bigrams)}")

print (f"les 10 premiers bi/tri grammes: {bigrams[:10]}")


Nombre de bigrammes/trigrammes (séparés par -): 6
les 10 premiers bi/tri grams: [('arc', 'en', 'ciel'), ('New', 'York'), ('wagon', 'restaurant'), ('chou', 'fleur'), ('arc', 'e'), ('n', 'ciel')]


In [38]:
def normalize_text(text):
    import unicodedata
    text = text.lower()
    
    text_nfd = unicodedata.normalize('NFD', text)
    text_without_accents = ''.join(char for char in text_nfd 
                                    if unicodedata.category(char) != 'Mn')
    nombres_texte = {
        '0': 'zero', '1': 'un', '2': 'deux', '3': 'trois', '4': 'quatre',
        '5': 'cinq', '6': 'six', '7': 'sept', '8': 'huit', '9': 'neuf'
    }
    
    text_normalized = ''
    for char in text_without_accents:
        if char.isdigit():
            text_normalized += nombres_texte[char] + ' '
        else:
            text_normalized += char
    
    return text_normalized.strip()

print("TEST \n")

sample_text = "Noël est le 25 décembre! C'est une fêtE avec Événements."
normalized = normalize_text(sample_text)

print(f"Original:  {sample_text}")
print(f"Normalisé: {normalized}")

text_normalized = normalize_text(text)
print(f"Nombre de caractères: {len(text)} → {len(text_normalized)}")
print(f"\nPremier 200 caractères normalisés:")
print(text_normalized[:200] + "...")


TEST 

Original:  Noël est le 25 décembre! C'est une fêtE avec Événements.
Normalisé: noel est le deux cinq  decembre! c'est une fete avec evenements.
Nombre de caractères: 1616 → 1634

Premier 200 caractères normalisés:
oh oh oh, noel arrive, et qui dit noel dit ... nourriture !
jeudi un huit  decembre prochain, a partir de midi, nous vous proposons un grand buffet collectif de noel pour feter ensemble vos departs en...


In [41]:
french_stopwords = {
    'le', 'la', 'les', 'de', 'des', 'un', 'une', 'et', 'ou', 'mais', 'donc',
    'car', 'par', 'pour', 'avec', 'sans', 'sur', 'sous', 'dans', 'entre',
    'avant', 'apres', 'pendant', 'lors', 'qui', 'que', 'quoi', 'quel', 'quelle',
    'dont', 'duquel', 'auquel', 'a', 'au', 'aux', 'ce', 'cet', 'cette', 'ces',
    'je', 'tu', 'il', 'elle', 'nous', 'vous', 'ils', 'elles', 'on', 'mon', 'ton',
    'son', 'notre', 'votre', 'leur', 'mes', 'tes', 'ses', 'nos', 'vos', 'leurs',
    'moi', 'toi', 'lui', 'me', 'te', 'se', 'nous', 'vous', 'leur', 'est', 'etre',
    'avoir', 'avoir', 'faire', 'aller', 'venir', 'pouvoir', 'devoir', 'vouloir',
    'c', 'est', 'd', 'l', 'j', 's', 't', 'n', 'm', 'y', 'en', 'a', 'as', 'oh'
}

def clean_tokens(tokens, remove_stopwords=True, remove_punctuation=True):
    import string
    
    cleaned = []
    
    for token in tokens:
        if remove_punctuation:
            if token in string.punctuation or all(c in string.punctuation for c in token):
                continue
            token = ''.join(char for char in token if char not in string.punctuation)
        
        if remove_stopwords:
            if token.lower() in french_stopwords:
                continue
        
        if token.strip():
            cleaned.append(token)
    
    return cleaned

print("netteoyage\n")

tokens_to_clean = tokenize_words(text_normalized)

print(f"Tokens avant nettoyage: {len(tokens_to_clean)}")
print(f"Premiers 20 tokens: {tokens_to_clean[:20]}\n")

cleaned_tokens = clean_tokens(tokens_to_clean, remove_stopwords=True, remove_punctuation=True)

print(f"Tokens après nettoyage: {len(cleaned_tokens)}")
print(f"Premiers 20 tokens nettoyés: {cleaned_tokens[:20]}\n")

print("STATS")
print(f"Tokens supprimés: {len(tokens_to_clean) - len(cleaned_tokens)}")
print(f"Ratio de compression: {(len(tokens_to_clean) - len(cleaned_tokens)) / len(tokens_to_clean) * 100:.1f}%")


netteoyage

Ressource NLTK manquante
Tokens avant nettoyage: 319
Premiers 20 tokens: ['oh', 'oh', 'oh', ',', 'noel', 'arrive', ',', 'et', 'qui', 'dit', 'noel', 'dit', '.', '.', '.', 'nourriture', '!', 'jeudi', 'un', 'huit']

Tokens après nettoyage: 156
Premiers 20 tokens nettoyés: ['noel', 'arrive', 'dit', 'noel', 'dit', 'nourriture', 'jeudi', 'huit', 'decembre', 'prochain', 'partir', 'midi', 'proposons', 'grand', 'buffet', 'collectif', 'noel', 'feter', 'ensemble', 'departs']

STATS
Tokens supprimés: 163
Ratio de compression: 51.1%


In [44]:

def simple_stemmer(word):
    suffixes = [
        'ement', 'ment', 'tion', 'sion',
        'able', 'ible', 'ateur', 'atrice',
        'eux', 'euse', 'ais', 'ait', 'aient',
        'és', 'é', 'e', 's',
        'et', 'elle', 'ette',
        'ment',
        'ant', 'ant', 'ent'
    ]
    
    word_lower = word.lower()
    
    for suffix in sorted(suffixes, key=len, reverse=True):
        if word_lower.endswith(suffix) and len(word_lower) > len(suffix) + 2:
            return word_lower[:-len(suffix)]
    
    return word_lower

def stem_words_list(words):
    return [simple_stemmer(word) for word in words]


def lemmatize_text_full(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas



print("STEMMING vs LEMMATISATION")

test_words = [
    'courant', 'courants', 'couru', 'courserait', 'coureur', 'coureurs',
    'marcher', 'marcheurs', 'marche', 'marchant', 'marches',
    'livre', 'livres', 'livreur', 'livreurs', 'livraison',
    'intelligent', 'intelligente', 'intelligents', 'intelligences',
    'rapidement', 'rapide', 'rapidité', 'rapidités'
]

print("\nTEST SUR DES MOTS SPÉCIFIQUES\n")
print(f"{'Mot Original':<20} | {'Stemming':<20} | {'Lemmatisation (spaCy)':<25}")

for word in test_words:
    stem = simple_stemmer(word)
    lemma_result = lemmatize_text_full(word)
    lemma = lemma_result[0] if lemma_result else word
    print(f"{word:<20} | {stem:<20} | {lemma:<25}")




STEMMING vs LEMMATISATION

TEST SUR DES MOTS SPÉCIFIQUES

Mot Original         | Stemming             | Lemmatisation (spaCy)    
courant              | cour                 | courir                   
courants             | courant              | courant                  
couru                | couru                | courir                   
courserait           | courser              | courser                  
coureur              | coureur              | coureur                  
coureurs             | coureur              | coureur                  
marcher              | marcher              | marcher                  
marcheurs            | marcheur             | marcheur                 
marche               | march                | marche                   
marchant             | march                | marchant                 
marches              | marche               | marches                  
livre                | livr                 | livre                    
livres

In [47]:
stemmed_tokens = stem_words_list(cleaned_tokens)

print(f"Tokens nettoyés: {len(cleaned_tokens)}")
print(f"Tokens après stemming: {len(stemmed_tokens)}")
print(f"Premiers 15 tokens nettoyés: {cleaned_tokens[:15]}")
print(f"Premiers 15 après stemming: {stemmed_tokens[:15]}\n")

lemmatized_tokens = lemmatize_text_full(text_normalized)
lemmatized_cleaned = [token.lemma_ for token in nlp(text_normalized) 
                      if token.text.lower() in [t.lower() for t in cleaned_tokens]]

print(f"Tokens après lemmatisation: {len(lemmatized_cleaned)}")
print(f"Premiers 30 après lemmatisation: {lemmatized_cleaned[:30]}\n")

unique_original = len(set(cleaned_tokens))
unique_stemmed = len(set(stemmed_tokens))
unique_lemmatized = len(set(lemmatized_cleaned))

print(f"\nMots uniques dans les tokens nettoyés: {unique_original}")
print(f"Mots uniques après stemming: {unique_stemmed} (réduction de {((unique_original - unique_stemmed) / unique_original * 100):.1f}%)")
print(f"Mots uniques après lemmatisation: {unique_lemmatized} (réduction de {((unique_original - unique_lemmatized) / unique_original * 100):.1f}%)")


Tokens nettoyés: 156
Tokens après stemming: 156
Premiers 15 tokens nettoyés: ['noel', 'arrive', 'dit', 'noel', 'dit', 'nourriture', 'jeudi', 'huit', 'decembre', 'prochain', 'partir', 'midi', 'proposons', 'grand', 'buffet']
Premiers 15 après stemming: ['noel', 'arriv', 'dit', 'noel', 'dit', 'nourritur', 'jeudi', 'huit', 'decembr', 'prochain', 'partir', 'midi', 'proposon', 'grand', 'buff']

Tokens après lemmatisation: 148
Premiers 30 après lemmatisation: ['noel', 'arriv', 'dire', 'noel', 'dire', 'nourriture', 'jeudi', 'huit', 'decembre', 'prochain', 'partir', 'midi', 'proposer', 'grand', 'buffet', 'collectif', 'noel', 'feter', 'ensemble', 'depart', 'vacance', 'hiver', 'premier', 'semestre', 'intense', 'voyage', 'disponible', 'servir', 'de', 'voir']


Mots uniques dans les tokens nettoyés: 126
Mots uniques après stemming: 125 (réduction de 0.8%)
Mots uniques après lemmatisation: 112 (réduction de 11.1%)


## Partie 2 

In [48]:
vocab = list(set(cleaned_tokens))  
vocab_sorted = sorted(vocab)

print(f"Vocabulaire créé à partir des tokens nettoyés")
print(f"  Taille du vocabulaire: {len(vocab)} mots uniques")
print(f"  Tokens nettoyés disponibles: {len(cleaned_tokens)}\n")

print(f"Premiers 20 mots du vocabulaire: {vocab_sorted[:20]}\n")
print(f"Derniers 20 mots du vocabulaire: {vocab_sorted[-20:]}")

Vocabulaire créé à partir des tokens nettoyés
  Taille du vocabulaire: 126 mots uniques
  Tokens nettoyés disponibles: 156

Premiers 20 mots du vocabulaire: ['admiration', 'afin', 'alerte', 'alternance', 'arc', 'arrive', 'aura', 'automatiquement', 'autour', 'beaux', 'bien', 'bienvenus', 'buffet', 'cadeau', 'cadeaux', 'cakes', 'camarade', 'campus', 'ceux', 'chacun']

Derniers 20 mots du vocabulaire: ['selon', 'semestre', 'sert', 'si', 'simple', 'sort', 'souhaitent', 'souhaitez', 'soyez', 'tartes', 'theme', 'tirage', 'tous', 'toutes', 'vacances', 'verrez', 'voyage', 'wagon', 'york', 'zero']


In [49]:
class OneHotEncoder:
    def __init__(self, vocabulary):

        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        print(f" OneHotEncoder initialisé avec {self.vocab_size} mots")
    
    def encode_word(self, word):
        if word not in self.vocabulary:
            raise ValueError(f"Mot '{word}' non trouvé dans le vocabulaire")

        one_hot_vector = [0] * self.vocab_size
        one_hot_vector[self.vocabulary[word]] = 1
        
        return one_hot_vector
    
    def encode_sentence(self, words):
        encoded = []
        for word in words:
            if word in self.vocabulary:
                encoded.append(self.encode_word(word))
        return encoded
    
    def decode_vector(self, vector):

        if vector.count(1) != 1:
            return None
        
        index = vector.index(1)
        for word, idx in self.vocabulary.items():
            if idx == index:
                return word
        return None
    
    def get_vocabulary_info(self):
        return {
            'size': self.vocab_size,
            'vocabulary': self.vocabulary
        }

if vocab:
    encoder = OneHotEncoder(vocab)
    
    test_word = vocab_sorted[0]
    print(f"Encodage du mot '{test_word}':")
    encoded = encoder.encode_word(test_word)
    print(f"  Vecteur: {encoded}")
    print(f"  Index du 1: {encoded.index(1)}\n")
    
    test_words = vocab_sorted[:5]
    print(f"Encodage des mots {test_words}:")
    for word in test_words:
        vec = encoder.encode_word(word)
        print(f"  {word:<15} → {vec.index(1):3d} (1 à la position {vec.index(1)})")
    
    print(f"\nDécodage des vecteurs:")
    for word in test_words:
        vec = encoder.encode_word(word)
        decoded = encoder.decode_vector(vec)
        print(f"  Vecteur → {decoded}")
    
    print(f"Taille du vocabulaire: {encoder.vocab_size} mots")
    print(f"Dimension de chaque vecteur one-hot: {encoder.vocab_size}")

 OneHotEncoder initialisé avec 126 mots
Encodage du mot 'admiration':
  Vecteur: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Index du 1: 0

Encodage des mots ['admiration', 'afin', 'alerte', 'alternance', 'arc']:
  admiration      →   0 (1 à la position 0)
  afin            →   1 (1 à la position 1)
  alerte          →   2 (1 à la position 2)
  alternance      →   3 (1 à la position 3)
  arc             →   4 (1 à la position 4)

Décodage des vecteurs:
  Vecteur → admiration
  Vecteur → afin
  Vecteur → alerte
  Vecteur → alternance
  Vecteur → arc
Taille du vocabulaire: 126 mots
Dimension de chaque vecteur one-hot: 126


In [51]:
class BagOfWords:    
    def __init__(self, vocabulary):
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        print(f"BagOfWords initialisé avec {self.vocab_size} mots")
    
    def encode_document(self, words):
        bow_vector = [0] * self.vocab_size
        
        for word in words:
            if word in self.vocabulary:
                idx = self.vocabulary[word]
                bow_vector[idx] += 1
        
        return bow_vector
    
    def encode_documents(self, documents):
        encoded_docs = []
        for doc in documents:
            encoded_docs.append(self.encode_document(doc))
        return encoded_docs
    
    def get_word_frequencies(self, vector):
        frequencies = {}
        for word, idx in self.vocabulary.items():
            if vector[idx] > 0:
                frequencies[word] = vector[idx]
        return frequencies
    
    def get_top_words(self, vector, top_n=10):
        frequencies = self.get_word_frequencies(vector)
        sorted_freq = sorted(frequencies.items(), key=lambda x: x[1], reverse=True)
        return sorted_freq[:top_n]
    
    def get_vocabulary_info(self):
        return {
            'size': self.vocab_size,
            'vocabulary': self.vocabulary
        }


if vocab:
    bow = BagOfWords(vocab)
    bow_vector = bow.encode_document(cleaned_tokens)
    
    print(f"Document: {len(cleaned_tokens)} mots")
    print(f"Vecteur BoW créé avec {bow.vocab_size} dimensions\n")
    
    frequencies = bow.get_word_frequencies(bow_vector)
    print(f"Nombre de mots uniques avec fréquence > 0: {len(frequencies)}\n")
    
    print("Top 15 mots les plus fréquents:")
    print("─" * 40)
    top_words = bow.get_top_words(bow_vector, 15)
    for word, freq in top_words:
        pourcentage = (freq / sum(bow_vector)) * 100
        barre = "█" * int(pourcentage / 2)
        print(f"{word:<15} | {freq:3d} | {pourcentage:5.2f}% {barre}")
    
    print(f"Vecteur BoW - Première moitié: {bow_vector[:10]}")
    print(f"Nombre total de mots: {sum(bow_vector)}")
    print(f"Moyenne d'occurrences par mot: {sum(bow_vector) / len(frequencies):.2f}")
    print(f"Sparsité: {(len([x for x in bow_vector if x == 0]) / len(bow_vector) * 100):.2f}% de zéros")

BagOfWords initialisé avec 126 mots
Document: 156 mots
Vecteur BoW créé avec 126 dimensions

Nombre de mots uniques avec fréquence > 0: 126

Top 15 mots les plus fréquents:
────────────────────────────────────────
noel            |   7 |  4.49% ██
du              |   3 |  1.92% 
ensemble        |   3 |  1.92% 
plus            |   3 |  1.92% 
bienvenus       |   2 |  1.28% 
collectif       |   2 |  1.28% 
decembre        |   2 |  1.28% 
dit             |   2 |  1.28% 
etes            |   2 |  1.28% 
grand           |   2 |  1.28% 
huit            |   2 |  1.28% 
jeudi           |   2 |  1.28% 
moment          |   2 |  1.28% 
offrir          |   2 |  1.28% 
prochain        |   2 |  1.28% 
Vecteur BoW - Première moitié: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Nombre total de mots: 156
Moyenne d'occurrences par mot: 1.24
Sparsité: 0.00% de zéros


In [ ]:
import numpy as np
from collections import defaultdict

class ContinuousBagOfWords:
    def __init__(self, vocabulary, window_size=2, embedding_dim=10):
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        np.random.seed(42)
        self.embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f" CBOW initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}")
        print(f"  Taille du contexte: ±{window_size} mots\n")
    
    def get_context_windows(self, words):
        windows = []
        for i in range(len(words)):
            start = max(0, i - self.window_size)
            end = min(len(words), i + self.window_size + 1)
            
            context = []
            for j in range(start, end):
                if j != i and words[j] in self.vocabulary:
                    context.append(words[j])
            
            if context and words[i] in self.vocabulary:
                windows.append((context, words[i]))
        
        return windows
    
    def get_embedding(self, word):
        if word in self.vocabulary:
            idx = self.vocabulary[word]
            return self.embeddings[idx]
        return None
    
    def get_context_vector(self, context_words):
        vectors = [self.get_embedding(w) for w in context_words if w in self.vocabulary]
        if vectors:
            return np.mean(vectors, axis=0)
        return np.zeros(self.embedding_dim)
    
    def predict_word_from_context(self, context_words, top_n=5):
        context_vec = self.get_context_vector(context_words)
        
        similarities = []
        for word, idx in self.vocabulary.items():
            sim = np.dot(context_vec, self.embeddings[idx])
            similarities.append((word, sim))
        
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_n]

class SkipGram:
    def __init__(self, vocabulary, window_size=2, embedding_dim=10):
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        np.random.seed(42)
        self.input_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        self.output_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f" Skip-Gram initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}")
        print(f"  Taille du contexte: ±{window_size} mots\n")
    
    def get_training_pairs(self, words):
        pairs = []
        for i in range(len(words)):
            if words[i] not in self.vocabulary:
                continue
            
            target = words[i]
            start = max(0, i - self.window_size)
            end = min(len(words), i + self.window_size + 1)
            
            for j in range(start, end):
                if j != i and words[j] in self.vocabulary:
                    pairs.append((target, words[j]))
        
        return pairs
    
    def get_embedding(self, word):
        if word in self.vocabulary:
            idx = self.vocabulary[word]
            return self.input_embeddings[idx]
        return None
    
    def predict_context(self, target_word, top_n=5):
        if target_word not in self.vocabulary:
            return []
        
        target_idx = self.vocabulary[target_word]
        target_vec = self.input_embeddings[target_idx]
        
        similarities = []
        for word, idx in self.vocabulary.items():
            sim = np.dot(target_vec, self.output_embeddings[idx])
            similarities.append((word, sim))
        
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_n]


class DistributedBagOfWords:
    def __init__(self, vocabulary, embedding_dim=20):
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.embedding_dim = embedding_dim
        
        np.random.seed(42)
        self.word_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f" Distributed Bag of Words initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}\n")
    
    def get_document_vector(self, words, doc_id=None):
        word_vecs = [self.word_embeddings[self.vocabulary[w]] 
                     for w in words if w in self.vocabulary]
        
        if word_vecs:
            doc_vec = np.mean(word_vecs, axis=0)
        else:
            doc_vec = np.zeros(self.embedding_dim)
        
        return doc_vec
    
    def get_document_similarity(self, doc1_words, doc2_words):
        vec1 = self.get_document_vector(doc1_words)
        vec2 = self.get_document_vector(doc2_words)
        
        norm1 = np.linalg.norm(vec1)
        norm2 = np.linalg.norm(vec2)
        
        if norm1 > 0 and norm2 > 0:
            return np.dot(vec1, vec2) / (norm1 * norm2)
        return 0.0

class DistributedMemory:
    def __init__(self, vocabulary, window_size=2, embedding_dim=20):
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        np.random.seed(42)
        self.word_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f" Distributed Memory initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}")
        print(f"  Taille du contexte: ±{window_size} mots\n")
    
    def get_context_windows_with_doc(self, words, doc_id=0):
        windows = []
        for i in range(len(words)):
            if words[i] not in self.vocabulary:
                continue
            
            start = max(0, i - self.window_size)
            end = min(len(words), i + self.window_size + 1)
            
            context = []
            for j in range(start, end):
                if j != i and words[j] in self.vocabulary:
                    context.append(words[j])
            
            if context:
                context_with_doc = context + [f"DOC_{doc_id}"]
                windows.append((context_with_doc, words[i]))
        
        return windows
    
    def get_document_representation(self, words, doc_id=0):
        word_vecs = [self.word_embeddings[self.vocabulary[w]] 
                     for w in words if w in self.vocabulary]
        
        if word_vecs:
            return np.mean(word_vecs, axis=0)
        return np.zeros(self.embedding_dim)

if vocab and cleaned_tokens:
    cbow = ContinuousBagOfWords(vocab, window_size=2, embedding_dim=10)
    skipgram = SkipGram(vocab, window_size=2, embedding_dim=10)
    dbow = DistributedBagOfWords(vocab, embedding_dim=20)
    dm = DistributedMemory(vocab, window_size=2, embedding_dim=20)
    
    sample_words = cleaned_tokens[:50]
    
    print("Prédit un mot à partir du contexte environnant\n")
    
    cbow_windows = cbow.get_context_windows(sample_words)
    print(f"Fenêtres de contexte extraites: {len(cbow_windows)}\n")

    if cbow_windows:
        context, target = cbow_windows[0]
        print(f"Exemple:")
        print(f"  Contexte: {context}")
        print(f"  Cible réelle: {target}")
        predictions = cbow.predict_word_from_context(context, top_n=5)
        print(f"  Top 5 prédictions:")
        for word, score in predictions:
            print(f"    {word:<15} score: {score:.4f}")

    print("Prédit le contexte à partir du mot cible\n")
    
    
 



    

 CBOW initialisé
  Vocabulaire: 126 mots
  Dimension des embeddings: 10
  Taille du contexte: ±2 mots

 Skip-Gram initialisé
  Vocabulaire: 126 mots
  Dimension des embeddings: 10
  Taille du contexte: ±2 mots

 Distributed Bag of Words initialisé
  Vocabulaire: 126 mots
  Dimension des embeddings: 20

 Distributed Memory initialisé
  Vocabulaire: 126 mots
  Dimension des embeddings: 20
  Taille du contexte: ±2 mots

Prédit un mot à partir du contexte environnant

Fenêtres de contexte extraites: 50

Exemple:
  Contexte: ['arrive', 'dit']
  Cible réelle: noel
  Top 5 prédictions:
    esprit          score: 2.5103
    intense         score: 2.3884
    mettez          score: 2.2822
    egalement       score: 1.9711
    repas           score: 1.9422
Prédit le contexte à partir du mot cible

Paires (target, contexte) créées: 194

Exemple:
  Mot cible: noel
  Contexte réel: arrive
  Top 5 contextes prédits:
    sapin           score: 3.7805
    partir          score: 3.2899
    dit          

In [58]:
if vocab and cleaned_tokens:
    skipgram_pairs = skipgram.get_training_pairs(sample_words)
    print(f"Paires (target, contexte) créées: {len(skipgram_pairs)}\n")

    if skipgram_pairs:
        target, context = skipgram_pairs[0]
        print(f"Exemple:")
        print(f"  Mot cible: {target}")
        print(f"  Contexte réel: {context}")
        predictions = skipgram.predict_context(target, top_n=5)
        print(f"  Top 5 contextes prédits:")
        for word, score in predictions:
            print(f"    {word:<15} score: {score:.4f}")

Paires (target, contexte) créées: 194

Exemple:
  Mot cible: noel
  Contexte réel: arrive
  Top 5 contextes prédits:
    sapin           score: 3.7805
    partir          score: 3.2899
    dit             score: 3.1178
    possibilites    score: 3.0251
    simple          score: 2.9209


In [56]:
if vocab and cleaned_tokens:
    
    print("Représente un document par un vecteur continu\n")
  
    doc1 = cleaned_tokens[:30]
    doc2 = cleaned_tokens[30:60]
    doc3 = cleaned_tokens[60:90]
    
    vec1 = dbow.get_document_vector(doc1)
    vec2 = dbow.get_document_vector(doc2)
    vec3 = dbow.get_document_vector(doc3)
    
    print(f"Vecteur document 1: {vec1[:5]}... (taille: {vec1.shape})")
    print(f"Vecteur document 2: {vec2[:5]}... (taille: {vec2.shape})")
    print(f"Vecteur document 3: {vec3[:5]}... (taille: {vec3.shape})\n")
    sim_1_2 = dbow.get_document_similarity(doc1, doc2)
    sim_1_3 = dbow.get_document_similarity(doc1, doc3)
    sim_2_3 = dbow.get_document_similarity(doc2, doc3)
    
    print("Similarités cosinus entre documents:")
    print(f"  Doc1 vs Doc2: {sim_1_2:.4f}")
    print(f"  Doc1 vs Doc3: {sim_1_3:.4f}")
    print(f"  Doc2 vs Doc3: {sim_2_3:.4f}")

    print("Combine contexte ET ID du document pour prédire\n")
    
    dm_windows = dm.get_context_windows_with_doc(sample_words[:30], doc_id=1)
    print(f"Fenêtres (contexte + doc_id, target) créées: {len(dm_windows)}\n")
    
    if dm_windows:
        context, target = dm_windows[0]
        print(f"Exemple:")
        print(f"  Contexte + DocID: {context}")
        print(f"  Cible: {target}")
    
    doc_repr = dm.get_document_representation(sample_words[:30], doc_id=1)
    print(f"  Représentation du doc: {doc_repr[:5]}... (taille: {doc_repr.shape})")

Représente un document par un vecteur continu

Vecteur document 1: [ 0.25120644  0.3022648   0.09942717 -0.09420201 -0.07429434]... (taille: (20,))
Vecteur document 2: [ 0.2168211   0.1556571  -0.06612796 -0.00911701 -0.11699125]... (taille: (20,))
Vecteur document 3: [ 0.3949813  -0.14876373  0.06921144 -0.03011026 -0.1838882 ]... (taille: (20,))

Similarités cosinus entre documents:
  Doc1 vs Doc2: 0.5054
  Doc1 vs Doc3: 0.3999
  Doc2 vs Doc3: 0.3598
Combine contexte ET ID du document pour prédire

Fenêtres (contexte + doc_id, target) créées: 30

Exemple:
  Contexte + DocID: ['arrive', 'dit', 'DOC_1']
  Cible: noel
  Représentation du doc: [ 0.25120644  0.3022648   0.09942717 -0.09420201 -0.07429434]... (taille: (20,))
